In [5]:
import duckdb
from pathlib import Path


duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open ("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

